In [3]:
from postgres import Postgres

pg = Postgres()

results = pg.query(
    """
 SELECT 
	cmetadata->'text' as text
    FROM langchain_pg_embedding 
	WHERE cmetadata @> '{"chunker": "sherpa"}' 
    AND LENGTH(cmetadata->>'sections'::TEXT) > 20
	LIMIT 5000;
                   
	"""
)
documents = [r[0] for r in results]

In [4]:
import spacy

import numpy as np

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


def remove_place_names_and_stop_words(text):

    custom_phrases = [
        "Abberley",
        "Abbots Langley",
        "Aberford",
        "Ab Kettleby Parish",
        "Acle",
        "Acton",
        "Addingham",
        "Adel",
        "Alcester",
        "Alfold",
        "Cleobury Mortimer",
        "Corby Glen",
        "Cossington",
        "Elmswell",
        "Fulbourn",
        "Nether Whitacre",
        "Sawtry",
        "Strensall & Towthorpe",
        "The Three Parishes",
        "Totnes",
        "West Wittering",
        "Whaley Bridge",
        "Winchfield",
        "Winkfield",
    ]

    doc = nlp(text)
    # Remove place names and stop words
    filtered_tokens = [
        token.text for token in doc if token.ent_type_ != "GPE" and not token.is_stop
    ]

    # Reconstruct the filtered text
    filtered_text = " ".join(filtered_tokens)

    # Remove custom stop phrases (case insensitive)
    filtered_text_lower = filtered_text.lower()
    for phrase in custom_phrases:
        filtered_text_lower = filtered_text_lower.replace(phrase.lower(), "")

    # Split and join to remove extra spaces
    filtered_text = " ".join(filtered_text_lower.split())

    return filtered_text


processed_documents = [remove_place_names_and_stop_words(doc) for doc in documents]

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Sample text fragments
texts = processed_documents

# Tokenize and get BERT embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Take the mean of token embeddings

embeddings = [get_embedding(text) for text in texts]